In [1]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [70]:
# Loading targets into stk from file.
stk_object.set_sim_time(days=18)
stk_object.Target_Loader("../../Input_Files/Targets_65.txt")
stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")

In [3]:
stk_object.Compute_AzEl()

In [4]:
stk_object.Generate_Pre_Planning_Data()
stk_object.Plan(1,20)

In [5]:
data_comparison = {}
data_comparison["Unplanned (%)"] = [np.count_nonzero(stk_object.target_bins[tar_num])/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Unplanned (Time)"] = [np.max(stk_object.target_times[tar_num])/86400 for tar_num in range(len(stk_object.targets))]
data_comparison["Planned (%)"] = [len(np.unique(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Planned (Time)"] = [max(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Time'].values)/86400 for tar_num in range(len(stk_object.targets))]

data_comparison = pd.DataFrame(data_comparison)
print(data_comparison)
print(data_comparison.describe())

In [ ]:
self = stk_object

az_range = list(range(0,360,10))
el_range = list(range(0,90,10))

self.Reset_Target_Bins()

# with alive_bar(len(self.targets)*len(self.satellites),force_tty=True,bar='classic',title='- Computing_Access',length=10) as bar:
#     for sat_num,sat in enumerate(self.satellites):
#         for tar_num,tar in enumerate(self.targets):
#             tar.GetAccessToObject(sat).ComputeAccess()
#             sat.GetAccessToObject(tar).ComputeAccess()
#             bar()

self.Pre_Planning_Hash_Map = {idx:{bin_num:[] for bin_num in range(324)} for idx in range(len(self.targets))}

with alive_bar(len(self.targets)*len(self.satellites),force_tty=True,bar='classic',title='- Getting_AzEl',length=10) as bar:
    for sat_num,sat in enumerate(self.satellites):
        for tar_num,tar in enumerate(self.targets):
            access = tar.GetAccessToObject(sat)
            Intervals = access.DataProviders.GetItemByName('AER Data').Group.Item(0).ExecElements(self.root.CurrentScenario.StartTime,
                                                                                        self.root.CurrentScenario.StopTime,
                                                                                        self.dt,['Time','Azimuth','Elevation']).Intervals
            start_times = []
            stop_times = []
            bins = []
            for Int in Intervals:
                times_az = Int.MultipleThresholdCrossings("Azimuth",az_range)
                times_el = Int.MultipleThresholdCrossings("Elevation",el_range)
                for i,t_az in enumerate(times_az[1:]):
                    for j,t_el in enumerate(times_el[1:]):
                        if len(t_az)>0 and len(t_el)>0:
                            for t in t_el:
                                if (any(t[0] <= value <= t[1] for value in t_az[0]) or 
                                    any(t_az[0][0] <= value <= t_az[0][1] for value in t)):
                                    self.Update_Target_Bins(t_az[0][0],i*9+j,tar_num)
                                    start_times.append(t_az[0][0]+.001)
                                    stop_times.append(t_az[0][1]-.001)
                                    bins.append(i*9+j)
                                    
            access = sat.GetAccessToObject(tar)
            res = access.DataProviders.GetItemByName('Sat Angles Data').ExecSingleElementsArray(start_times,['Cross Track','Along Track'])
            crosstrack = res.GetArray(0)
            alongtrack = res.GetArray(1)
            for b,t,ct,at in zip(bins,start_times,crosstrack,alongtrack):
                self.Pre_Planning_Hash_Map[tar_num][b].append([t,ct,at,sat_num])
            
            res = access.DataProviders.GetItemByName('Sat Angles Data').ExecSingleElementsArray(stop_times,['Cross Track','Along Track'])
            crosstrack = res.GetArray(0)
            alongtrack = res.GetArray(1)
            for b,t,ct,at in zip(bins,start_times,crosstrack,alongtrack):
                self.Pre_Planning_Hash_Map[tar_num][b].append([t,ct,at,sat_num])
            
            bar()
            
with alive_bar(len(self.targets)*324,force_tty=True,bar='classic',title='- Sorting_Data',length=10) as bar:
    for tar_num in range(len(self.targets)):
        for bin_num in range(324):
            a = np.array(self.Pre_Planning_Hash_Map[tar_num][bin_num],dtype=float)
            if len(a) > 0:
                self.Pre_Planning_Hash_Map[tar_num][bin_num] = a[a[:,0].argsort()]
            bar()

In [ ]:
stk_object.Plan(1,20)

data_comparison = {}
data_comparison["Unplanned (%)"] = [np.count_nonzero(stk_object.target_bins[tar_num])/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Unplanned (Time)"] = [np.max(stk_object.target_times[tar_num])/86400 for tar_num in range(len(stk_object.targets))]
data_comparison["Planned (%)"] = [len(np.unique(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Planned (Time)"] = [np.max(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Time'].values)/86400 for tar_num in range(len(stk_object.targets))]

data_comparison = pd.DataFrame(data_comparison)
print(data_comparison)
print(data_comparison.describe())

In [ ]:
fig = go.Figure()

res = access.DataProviders.GetItemByName('Sat Angles Data').ExecElements(start_times,stop_times,60,['Time','Cross Track','Along Track'])
df = res.DataSets.ToPandasDataFrame()
df.dropna()

fig.add_trace(go.Scatter(x=df['time'],y=df['cross track'],mode="markers"))

res = access.DataProviders.GetItemByName('Sat Angles Data').ExecSingleElementsArray(start_times,['Cross Track','Along Track'])
crosstrack = res.GetArray(0)
alongtrack = res.GetArray(1)
fig.add_trace(go.Scatter(x=start_times,y=crosstrack,mode="markers"))

res = access.DataProviders.GetItemByName('Sat Angles Data').ExecSingleElementsArray(stop_times,['Cross Track','Along Track'])
crosstrack = res.GetArray(0)
alongtrack = res.GetArray(1)
fig.add_trace(go.Scatter(x=stop_times,y=crosstrack,mode="markers"))

fig.show()